In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
raw_user = pd.read_csv('./data/train_users_2.csv')
raw_session = pd.read_csv('./data/sessions.csv')
raw_countries = pd.read_csv('./data/countries.csv')
raw_age_gender_bkts = pd.read_csv('./data/age_gender_bkts.csv')

In [7]:
raw_age_gender_bkts.head(10)

,age_bucket,country_destination,gender,population_in_thousands,year
0,100+,AU,male,1.0,2015.0
1,95-99,AU,male,9.0,2015.0
2,90-94,AU,male,47.0,2015.0
3,85-89,AU,male,118.0,2015.0
4,80-84,AU,male,199.0,2015.0
5,75-79,AU,male,298.0,2015.0
6,70-74,AU,male,415.0,2015.0
7,65-69,AU,male,574.0,2015.0
8,60-64,AU,male,636.0,2015.0
9,55-59,AU,male,714.0,2015.0


In [8]:
raw_age_gender_bkts.shape

(420, 5)

In [9]:
raw_age_gender_bkts.columns

Index(['age_bucket', 'country_destination', 'gender',
       'population_in_thousands', 'year'],
      dtype='object')